In [ ]:
# imports and Setup
import itertools
import os
import time
import numpy as np
from datetime import datetime
from scipy.spatial.distance import pdist
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.ProcessMiningTasks.LogGenerator.ASP.ASPLogGenerator import AspGenerator
from ga_objects.utils.encoder import Encoder
from ga_objects.utils.tools import Tools


In [ ]:
# configurations

# number of runs
num_runs = 1

# parameters for trace generation
pop_list = [10]                      # size of the population
num_event_list = [30]                 # choose from 30, 50, 70, 90
declare_model_list = ["model1"]       # choose from model1, model2, model3, model4


In [ ]:
# setup output paths

current_date = datetime.today().strftime('%m-%d-%H-%M')
base_dir = os.getcwd()
results_dir = os.path.join(base_dir, "results", current_date)
os.makedirs(results_dir, exist_ok=True)

result_file = os.path.join(results_dir, "result.csv")


In [ ]:
# run ASP generation and encoding

with open(result_file, "w") as f:
    f.write("ID,Run,Model,PopulationSize,TraceLength,ExecTime,Diversity\n")

    for run in range(1, num_runs + 1):
        ID = 1
        for pop_size, trace_length, model_name in itertools.product(pop_list, num_event_list, declare_model_list):

            model_path = os.path.join("..", "declare_models", model_name, f"{model_name}.decl")
            model: DeclareModel = DeclareModel().parse_from_file(model_path)

            print(f"\n🔧 Run {run} - ID {ID}: {model_name} | {trace_length} events | {pop_size} traces")

            asp_gen = AspGenerator(model, pop_size, trace_length, trace_length, verbose=False)

            start_time = time.time()
            asp_gen.run()
            exec_time = time.time() - start_time

            asp_gen.to_xes("ASP")

            encoder = Encoder(model.activities)
            traces = Tools.extract_traces_from_xes("ASP.xes", trace_length)
            encoded_traces = [encoder.encode(trace) for trace in traces]

            population = np.array(encoded_traces)
            dist_matrix = pdist(population, metric='hamming')
            diversity_score = dist_matrix.mean()

            f.write(f"{ID},{run},{model_name},{pop_size},{trace_length},{exec_time:.4f}s,{diversity_score:.6f}\n")
            print(f"Done in {exec_time:.2f}s | Diversity: {diversity_score:.4f}")

            ID += 1


In [ ]:
# show results path

print(f"\nResults saved to: {result_file}")
